### Recommendation System


In [1]:
import math

# Content-based: Movies dataset
movies = [
    {"title": "Inception", "genre": "Sci-Fi", "director": "Nolan"},
    {"title": "Interstellar", "genre": "Sci-Fi", "director": "Nolan"},
    {"title": "The Dark Knight", "genre": "Action", "director": "Nolan"},
    {"title": "The Shining", "genre": "Horror", "director": "Kubrick"},
    {"title": "It", "genre": "Horror", "director": "Muschietti"},
]

# Collaborative filtering: User ratings
user_ratings = {
    "Alice": {"Inception": 5, "Interstellar": 4, "The Shining": 2},
    "Bob": {"Inception": 4, "The Dark Knight": 5, "It": 3},
    "Carol": {"Interstellar": 5, "The Shining": 4, "It": 4},
    "Dave": {"Inception": 2, "Interstellar": 5, "The Dark Knight": 4}
}


####
Content-Based Filtering (Movies by Genre + Director

In [4]:
# Convert a movie into a feature vector (genre + director)
def movie_vector(movie):
    return [movie["genre"], movie["director"]]

# Cosine similarity for features
def cosine_similarity_features(f1, f2):
    all_features = list(set(f1 + f2))
    vec1 = [1 if feat in f1 else 0 for feat in all_features]
    vec2 = [1 if feat in f2 else 0 for feat in all_features]
    
    dot = sum(a*b for a, b in zip(vec1, vec2))
    mag1 = math.sqrt(sum(a*a for a in vec1))
    mag2 = math.sqrt(sum(b*b for b in vec2))
    
    if mag1 == 0 or mag2 == 0:
        return 0
    return dot / (mag1 * mag2)

# Recommend similar movies
def recommend_movies_content(target_title, top_n=2):
    target_movie = next(movie for movie in movies if movie["title"] == target_title)
    sims = []
    
    for movie in movies:
        if movie["title"] == target_title:
            continue
        sim = cosine_similarity_features(movie_vector(target_movie), movie_vector(movie))
        sims.append((sim, movie["title"]))
    
    sims.sort(reverse=True)
    return [title for _, title in sims[:top_n]]

print("Content-based recommendations for 'Inception':", recommend_movies_content("Inception"))
print("Content-based recommendations for 'The Shining':", recommend_movies_content("The Shining"))



Content-based recommendations for 'Inception': ['Interstellar', 'The Dark Knight']
Content-based recommendations for 'The Shining': ['It', 'The Dark Knight']


#### Collaborative Filtering (Movies by Ratings)

In [5]:
# Cosine similarity between users
def cosine_similarity_users(u1, u2):
    common_movies = set(u1.keys()) & set(u2.keys())
    if not common_movies:
        return 0
    
    vec1 = [u1[m] for m in common_movies]
    vec2 = [u2[m] for m in common_movies]
    
    dot = sum(a*b for a, b in zip(vec1, vec2))
    mag1 = math.sqrt(sum(a*a for a in vec1))
    mag2 = math.sqrt(sum(b*b for b in vec2))
    
    if mag1 == 0 or mag2 == 0:
        return 0
    return dot / (mag1 * mag2)

# Predict rating for a user and a movie
def predict_rating(user, movie, k=2):
    sims = []
    for other_user, ratings in user_ratings.items():
        if other_user == user or movie not in ratings:
            continue
        sim = cosine_similarity_users(user_ratings[user], ratings)
        sims.append((sim, ratings[movie]))
    
    sims.sort(reverse=True)
    top_k = sims[:k]
    if not top_k:
        return None
    
    numerator = sum(sim*rating for sim, rating in top_k)
    denominator = sum(sim for sim, _ in top_k)
    return numerator / denominator if denominator != 0 else None

# Recommend new movies to a user
def recommend_movies_collab(user, top_n=2):
    all_movies = set(m for r in user_ratings.values() for m in r)
    unrated = [m for m in all_movies if m not in user_ratings[user]]
    
    predictions = [(predict_rating(user, m), m) for m in unrated]
    predictions = [(p, m) for p, m in predictions if p is not None]
    predictions.sort(reverse=True)
    
    return [m for _, m in predictions[:top_n]]


print("Collaborative filtering recommendations for Alice:", recommend_movies_collab("Alice"))
print("Collaborative filtering recommendations for Bob:", recommend_movies_collab("Bob"))
